# Implementing an untyped Lambda Calculus


To introduce `metadsl` we will show how to create a working version of the untyped lambda calculus.

This will let us create the some basic combinators, verify that they before as expected under equiavelency.

Then we can also create an implementation of arithmetic on natural numbers, using church encoding and show how to convert
between these and Python's integers.


## Terms
The lambda calculus might sound abstract and odd, but if you know how to write functions in Python you already have mastered it!
The idea is a very simple language that only has functions that take in one argument.
Nothing, else, which seems a bit silly maybe, and it is not very useful by itself, but when it is integrated into a system with other data
types, it becomes a useful so that we can represent functions.


First, let's create a concept of a "variable" that can optionally have a name. It should only be equal to itself, so that we don't have to worry about shadowing variable names:

In [1]:
from __future__ import annotations

import dataclasses
import typing


@dataclasses.dataclass(eq=False)
class Variable:
    name: typing.Optional[str] = None
    
    
    def __str__(self):
        return self.name or str(id(self))
        
    def __repr__(self):
        return f"Variable({str(self)})"
        
x1 = Variable("x")
x2 = Variable("x")

assert x1 == x1
assert x1 != x2
assert hash(x1) != hash(x2)

Now, let's build our one and only key abstraction, a lambda term.
You should be able to call it and also create it by having a body and a variable. 

In [2]:
import metadsl

class LambdaTerm(metadsl.Expression):
    @metadsl.expression
    def __call__(self, arg: LambdaTerm) -> LambdaTerm:
        ...

@metadsl.expression
def abstraction(variable: Variable, body: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.expression
def from_variable(variable: Variable) -> LambdaTerm:
    ...

def create_abstraction(fn: typing.Callable[[LambdaTerm], LambdaTerm]) -> LambdaTerm:
    arg = Variable()
    return abstraction(arg, fn(from_variable(arg)))

We can construct the identity function

In [3]:
@create_abstraction
def identity(x):
    return x

In [4]:
# NBVAL_IGNORE_OUTPUT
identity

<class '__main__.LambdaTerm'>(abstraction, (Variable(4477822512), <class '__main__.LambdaTerm'>(from_variable, (Variable(4477822512),), {})), {})

and then call it on a term:

In [5]:
# NBVAL_IGNORE_OUTPUT
y_term = from_variable(Variable("y"))
called_identity = identity(y_term)
called_identity

<class '__main__.LambdaTerm'>(LambdaTerm.__call__, (<class '__main__.LambdaTerm'>(abstraction, (Variable(4477822512), <class '__main__.LambdaTerm'>(from_variable, (Variable(4477822512),), {})), {}), <class '__main__.LambdaTerm'>(from_variable, (Variable(y),), {})), {})

## Pretty Printing

This is a bit of a verbose way to print these.

Let's create a custom function that walks the tree and converts them to a string:

In [6]:
pretty_print_rules = metadsl.RulesRepeatFold()


@metadsl.expression
def string_term(s: str) -> LambdaTerm:
    ...


@pretty_print_rules.append
@metadsl.rule
def pp_variable(variable: Variable):
    return (
        from_variable(variable),
        lambda: string_term(str(variable))
    )

@pretty_print_rules.append
@metadsl.rule
def pp_abstraction(variable: Variable, body: str):
    return (
        abstraction(variable, string_term(body)),
        lambda: string_term(f"(λ{variable}.{body})")
    )


@pretty_print_rules.append
@metadsl.rule
def pp_application(fn: str, arg: str):
    return (
        string_term(fn)(string_term(arg)),
        lambda: string_term(f"({fn} {arg})")
    )

@metadsl.expression
def unbox_str(t: LambdaTerm) -> str:
    ...


@pretty_print_rules.append
@metadsl.rule
def pp_unbox(s: str):
    return unbox_str(string_term(s)), lambda: s

In [7]:
# NBVAL_IGNORE_OUTPUT
metadsl.execute_rule(pretty_print_rules, unbox_str(called_identity))

'((λ4477822512.4477822512) y)'

## Beta Reduction

To make this useful, we need to define some semantics on these.

What's a useful thing to do with the lambda calculus?

Well it would be nice to actually be able to "compute" these values by replacing all instances of a variable with it's argument.

So let's replace all instances of calling a variable by replacing the arg inside of it:

In [8]:
beta_reduce_rules = metadsl.RulesRepeatFold()

@beta_reduce_rules.append
@metadsl.rule
def _beta_reduce(var: Variable, body: LambdaTerm, arg: LambdaTerm):
    def replacement():
        # replaces all instances of var with arg
        replace_rules = metadsl.RulesRepeatFold(
            metadsl.rule(lambda: (from_variable(var), lambda: arg))
        )
        return metadsl.execute_rule(replace_rules, body) or body

    return abstraction(var, body)(arg), replacement

metadsl.execute_rule(beta_reduce_rules, called_identity)

<class '__main__.LambdaTerm'>(from_variable, (Variable(y),), {})

Now let's combine these two, to execute them in sequence. First we want to beta reduce, until there is no more to beta reduce, then
we wanna turn that into a pretty print representation:

In [9]:
rules = metadsl.RulesRepeatSequence(beta_reduce_rules, pretty_print_rules)
def execute(expr: LambdaTerm) -> str:
    return metadsl.execute_rule(rules, unbox_str(expr))

In [10]:
print(execute(called_identity))

y


## Alpha Conversion

This looks allright, but before we do anything serious, let's deal with the annoying issue of these variable names. They aren't much fun to read! Let's fix this by implementing [α-conversion](https://en.wikipedia.org/wiki/Lambda_calculus#%CE%B1-conversion). The basic idea is that given some lambda expression, we rename all the variables to make them standard. Let's create a bunch of global variables, and then create a function that will transform an expression into a normal form using them:

In [11]:
GLOBAL_VARS = [Variable(chr(i)) for i in range(ord('a'), ord('a') + 100)]
CURRENT_GLOBAL_VAR = 0

In [12]:
GLOBAL_VARS[0]

Variable(a)

The idea is that we will map each variable we find in the expression to one of the global variables:

In [13]:
alpha_convert_rules = metadsl.RulesRepeatFold()

def execute_alpha_convert(x):
    global CURRENT_GLOBAL_VAR
    CURRENT_GLOBAL_VAR = 0
    return metadsl.execute_rule(alpha_convert_rules, x)


@alpha_convert_rules.append
@metadsl.rule
def _alpha_convert(var: Variable, body: LambdaTerm):
    def replacement():
        if var in GLOBAL_VARS: 
            raise metadsl.NoMatch
        global CURRENT_GLOBAL_VAR

        new_var = GLOBAL_VARS[CURRENT_GLOBAL_VAR]
        CURRENT_GLOBAL_VAR += 1

        replace_rules = metadsl.RulesRepeatFold(
            metadsl.rule(lambda: (var, new_var))
        )

        res = abstraction(new_var, metadsl.execute_rule(replace_rules, body))

        # We have already replaced this if we got the same
        if res == abstraction(var, body):
            raise metadsl.NoMatch
        return res

    return abstraction(var, body), replacement

In [14]:
# NBVAL_IGNORE_OUTPUT
identity

<class '__main__.LambdaTerm'>(abstraction, (Variable(4477822512), <class '__main__.LambdaTerm'>(from_variable, (Variable(4477822512),), {})), {})

In [15]:
metadsl.execute_rule(alpha_convert_rules, identity)

<class '__main__.LambdaTerm'>(abstraction, (Variable(a), <class '__main__.LambdaTerm'>(from_variable, (Variable(a),), {})), {})

Now let's add this to our execution step, so that we print before and after beta reducing:

In [16]:
core_replacements = metadsl.RulesRepeatSequence(beta_reduce_rules)
def execute(expr: LambdaTerm) -> str:
    expr = execute_alpha_convert(expr)
    res = unbox_str(expr)
    return f"{metadsl.execute_rule(pretty_print_rules, res)} ⇒ {metadsl.execute_rule(metadsl.RuleInOrder(core_replacements, pretty_print_rules), res)}"


In [17]:
execute_alpha_convert(identity)

<class '__main__.LambdaTerm'>(abstraction, (Variable(a), <class '__main__.LambdaTerm'>(from_variable, (Variable(a),), {})), {})

In [18]:
execute(identity(identity))

'((λa.a) (λb.b)) ⇒ ((λa.a) (λb.b))'

Now we are finally getting somewhere!

In [19]:
execute(identity(identity)(from_variable(Variable("Input"))))

'(((λa.a) (λb.b)) Input) ⇒ (((λa.a) (λb.b)) Input)'

Let's actually just set it to always pretty print this representation:

In [20]:
LambdaTerm.__repr__ = execute

In [21]:
identity

(λa.a) ⇒ (λa.a)

In [22]:
identity(identity)

((λa.a) (λb.b)) ⇒ ((λa.a) (λb.b))

## Church Numerals

Now that we have a working system, we can start to implement some more useful programming on top of it.

One thing we can do is [represent natural numbers](https://en.wikipedia.org/wiki/Church_encoding#Church_numerals) with lambda expression and do math on them.

### Python Ints
First, let's add the ability to embed Python's ints inside a function:

In [23]:
@metadsl.expression
def int_term(i: int) -> LambdaTerm:
    ...

In [24]:
int_term(123)

unbox_str(int_term(123)) ⇒ unbox_str(int_term(123))

Oops! We don't know how to turn a int represenation of a lambda term into a string, lets fix that:

In [25]:
@pretty_print_rules.append
@metadsl.rule
def int_to_str(i: int):
    return int_term(i), lambda: string_term(str(i))

In [26]:
identity(int_term(123))

((λa.a) 123) ⇒ ((λa.a) 123)

### Church Numerals

How do we create a church numeral? Well we have a function that calls itself $n$ times, to represent $n$:

In [27]:
def church_numeral(n: int) -> LambdaTerm:
    @create_abstraction
    def outer(f):
        
        @create_abstraction
        def inner(x):
            for i in range(n):
                x = f(x)
            return x
        return inner
    return outer

In [28]:
church_numeral(0)

(λa.(λb.b)) ⇒ (λa.(λb.b))

In [29]:
x = church_numeral(1)
x

(λa.(λb.(a b))) ⇒ (λa.(λb.(a b)))

In [30]:
church_numeral(2)

(λa.(λb.(a (a b)))) ⇒ (λa.(λb.(a (a b))))

In [31]:
church_numeral(3)

(λa.(λb.(a (a (a b))))) ⇒ (λa.(λb.(a (a (a b)))))

Now let's add some replacements that from python integers to church numerals:

In [32]:
to_church_rules = metadsl.RulesRepeatFold()

@to_church_rules.append
@metadsl.rule
def int_to_abstraction(i: int):
    return int_term(i), lambda: church_numeral(i)

In [33]:
metadsl.execute_rule(to_church_rules, int_term(10))

(λa.(λb.(a (a (a (a (a (a (a (a (a (a b)))))))))))) ⇒ (λa.(λb.(a (a (a (a (a (a (a (a (a (a b))))))))))))

### Math

Add finally we can define math on these church numerals:

In [34]:
@metadsl.expression
def add(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.expression
def succ(x: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.expression
def mult(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

@metadsl.expression
def exp(x: LambdaTerm, y: LambdaTerm) -> LambdaTerm:
    ...

#### on Python Integers
First, we can define how these functions behave on Python integers:

In [35]:
int_math_rules = metadsl.RulesRepeatFold()


@int_math_rules.append
@metadsl.rule
def _add_ints(x: int, y: int):
    return add(int_term(x), int_term(y)), lambda: int_term(x + y)

@int_math_rules.append
@metadsl.rule
def _succ_int(x: int):
    return succ(int_term(x)), lambda: int_term(x + 1)

@int_math_rules.append
@metadsl.rule
def _mult_int(x: int, y: int):
    return mult(int_term(x), int_term(y)), lambda: int_term(x * y)


@int_math_rules.append
@metadsl.rule
def _exp_int(x: int, y: int):
    return exp(int_term(x), int_term(y)), lambda: int_term(x ** y)

Now we can add these to our core replacements that we defined above and our executed during printing:

In [36]:
core_replacements.append(int_math_rules)
pass

In [37]:
add(int_term(1), int_term(2))

unbox_str(add(string_term(1), string_term(2))) ⇒ unbox_str(add(string_term(1), string_term(2)))

Oh, we forgot to define the pretty printing version of these operations. Let's do that now as well:

In [38]:
@pretty_print_rules.append
@metadsl.rule
def _add_str(x: str, y: str):
    return add(string_term(x), string_term(y)), lambda: string_term(f"{x} + {y}")

@pretty_print_rules.append
@metadsl.rule
def _succ_str(x: str):
    return succ(string_term(x)), lambda: string_term(f"succ({x})")

@pretty_print_rules.append
@metadsl.rule
def _mult_str(x: str, y: str):
    return mult(string_term(x), string_term(y)), lambda: string_term(f"{x} * {y}")

@pretty_print_rules.append
@metadsl.rule
def _exp_str(x: str, y: str):
    return exp(string_term(x), string_term(y)), lambda: string_term(f"{x}^{y}")

In [39]:
add(int_term(1), int_term(2))

1 + 2 ⇒ 1 + 2

Sweet!

#### on Church numerals
But what if we turn them into church numerals first?

In [41]:
metadsl.execute_rule(to_church_rules, add(int_term(1), int_term(2)))

(λa.(λb.(a b))) + (λc.(λd.(c (c d)))) ⇒ (λa.(λb.(a b))) + (λc.(λd.(c (c d))))

Well we don't know how to add those. First, let's define the operations as abstractions themselves (following wikipedia here on the naming):

In [42]:
@create_abstraction
def add_abstraction(m):
    @create_abstraction
    def inner(n):
        @create_abstraction
        def inner(f):
            @create_abstraction
            def inner(x):
                return m(f)(n(f)(x))
            return inner
        return inner
    return inner

@create_abstraction
def succ_abstraction(n):
    @create_abstraction
    def inner(f):
        @create_abstraction
        def inner(x):
            return f(n(f)(x))
        return inner
    return inner

@create_abstraction
def mult_abstraction(m):
    @create_abstraction
    def inner(n):
        @create_abstraction
        def inner(f):
            return m(n(f))
        return inner
    return inner

@create_abstraction
def exp_abstraction(m):
    @create_abstraction
    def inner(n):
        return n(m)
    return inner


Is this right? Well let's try it out!

In [44]:
zero = metadsl.execute_rule(to_church_rules, int_term(0))
zero

(λa.(λb.b)) ⇒ (λa.(λb.b))

In [45]:
add_abstraction(zero)(zero)

(((λa.(λb.(λc.(λd.((a c) ((b c) d)))))) (λe.(λf.f))) (λg.(λh.h))) ⇒ (((λa.(λb.(λc.(λd.((a c) ((b c) d)))))) (λe.(λf.f))) (λg.(λh.h)))

In [46]:
zero

(λa.(λb.b)) ⇒ (λa.(λb.b))

Sweet! Those look same, that's good. 

And we should find that adding one is the same as the succesor functions...

In [48]:
one = metadsl.execute_rule(to_church_rules, int_term(1))

In [49]:
add_abstraction(one)

((λa.(λb.(λc.(λd.((a c) ((b c) d)))))) (λe.(λf.(e f)))) ⇒ ((λa.(λb.(λc.(λd.((a c) ((b c) d)))))) (λe.(λf.(e f))))

In [50]:
succ_abstraction

(λa.(λb.(λc.(b ((a b) c))))) ⇒ (λa.(λb.(λc.(b ((a b) c)))))

Those look the same! Nice!

OK What about multiplying by zero, that should always return zero...

In [51]:
mult_abstraction(zero)(from_variable(Variable("doesnt matter")))

(((λa.(λb.(λc.(a (b c))))) (λd.(λe.e))) doesnt matter) ⇒ (((λa.(λb.(λc.(a (b c))))) (λd.(λe.e))) doesnt matter)

Yep!

It's kind cool how exponentiation is now so simple... Let's try raising something to the 1st power, this should give us back itself:

In [52]:
exp_abstraction

(λa.(λb.(b a))) ⇒ (λa.(λb.(b a)))

In [53]:
exp_abstraction(from_variable(Variable("doesnt matter")))(one)

(((λa.(λb.(b a))) doesnt matter) (λc.(λd.(c d)))) ⇒ (((λa.(λb.(b a))) doesnt matter) (λc.(λd.(c d))))

This is actually the same as  the input, despite it being wrapped in another function. If we had implemented [Eta-conversion](https://en.wikipedia.org/wiki/Lambda_calculus#%CE%B7-conversion) this would reduce to "doesn't matter".

Now we can map these to the operations we defined above:

In [54]:
church_math_rules = metadsl.RulesRepeatFold()

@church_math_rules.append
@metadsl.rule
def _add(x: LambdaTerm, y: LambdaTerm):
    return add(x, y), lambda: add_abstraction(x)(y)

@church_math_rules.append
@metadsl.rule
def _mult(x: LambdaTerm, y: LambdaTerm):
    return mult(x, y), lambda: mult_abstraction(x)(y)

@church_math_rules.append
@metadsl.rule
def _exp(x: LambdaTerm, y: LambdaTerm):
    return exp(x, y), lambda: exp_abstraction(x)(y)


In [55]:
add(zero, one)

(λa.(λb.b)) + (λc.(λd.(c d))) ⇒ (λa.(λb.b)) + (λc.(λd.(c d)))

In [56]:
one

(λa.(λb.(a b))) ⇒ (λa.(λb.(a b)))

In [57]:
add(int_term(10), int_term(5))

10 + 5 ⇒ 10 + 5

In [60]:
metadsl.execute_rule(church_math_rules, metadsl.execute_rule(to_church_rules, add(int_term(10), int_term(5))))

(((λa.(λb.(λc.(λd.((a c) ((b c) d)))))) (λe.(λf.(e (e (e (e (e (e (e (e (e (e f))))))))))))) (λg.(λh.(g (g (g (g (g h)))))))) ⇒ (((λa.(λb.(λc.(λd.((a c) ((b c) d)))))) (λe.(λf.(e (e (e (e (e (e (e (e (e (e f))))))))))))) (λg.(λh.(g (g (g (g (g h))))))))

So now we have developed al little lambda calculus calculator. We can define custom pipelines now, reausing all of this work, to apply the steps in any order we would like.

And if we have users, they can right functions with the functions, and not worry how they are compiled. For example, I could write a function like this:

In [61]:
def my_special_math(x, y, z):
    return mult(add(x, y), z)

And the function is totally seperate from what we call it with, or how it ends up being compiled:

In [62]:
my_special_math(int_term(10), int_term(10), int_term(10))

10 + 10 * 10 ⇒ 10 + 10 * 10

In [63]:
my_special_math(zero, one, one)

(λa.(λb.b)) + (λc.(λd.(c d))) * (λe.(λf.(e f))) ⇒ (λa.(λb.b)) + (λc.(λd.(c d))) * (λe.(λf.(e f)))

In [64]:
metadsl.execute_rule(church_math_rules, metadsl.execute_rule(to_church_rules, my_special_math(zero, int_term(10), int_term(10))))

RecursionError: maximum recursion depth exceeded while calling a Python object

For another tutorial, it would be useful to show how this compares to a *typed* lambda calculus. That would let us deal with functions over multiple data types, so we know if operations like addition even make sense.